# Task

Keep using the One-piece [competition case](https://www.kaggle.com/t/f5f7783abf31495f9593b3d93a18f9eb).

This time, it is based on the multiple linear regression framework as in the Assignment 2 (yes, as a TV series).
$$y=x'\beta + \epsilon.$$


1. Have you transformed any variables?
Polynomial transformation? Interactions?
1. Week 6 will talk about LOWESS, a simple nonparametric method.
1. Model averaging (or model assembly in ML language, stacking in particular). If you have a few competitive models, sometimes, the average of their predictions is better than any single model's prediction.
1. Get the test sample for prediction and submit your results on Kaggle to get your Kaggle score screenshot. Show the screenshot in the PDF file.

**Note:**
- All instructions in Assignment 1 apply.
- Use $\leq 300$ words.

## Install Packages

In [96]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import  Lasso, LinearRegression,LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from itertools import combinations
from sklearn.metrics import mean_squared_error

## 1. Read Data

In [97]:
train_url = "https://github.com/joshcpld/ada/raw/main/Assignment%202/data/train_data.csv"
train_df = pd.read_csv(train_url)
test_url = "https://github.com/joshcpld/ada/raw/main/Assignment%202/data/test_data.csv"
test_df = pd.read_csv(test_url)

train_df.head()

,ID,Y,X1,X2,X3,X4,X5,X6,X7,X8,...,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50
0,0,-1.399091,1.174139,1.413109,0.164693,-1.067338,0.015324,-1.280970,0.489681,-0.371982,...,-0.115044,-2.580043,-0.812428,0.772820,-0.460444,0.190422,-0.362052,-1.119038,0.916313,-1.517434
1,1,3.097990,0.208922,0.931231,0.838779,0.893483,-0.510555,0.900289,-0.042490,0.839400,...,1.155635,0.673035,-0.438152,-0.001316,-0.761800,1.335092,0.901978,-1.549504,-0.456224,0.223405
2,2,-1.707346,-0.744982,0.962118,0.615392,-0.427943,-0.014912,1.138781,1.159491,0.055467,...,0.299277,1.387495,-0.007519,-0.464825,0.830986,0.373124,0.319232,-0.577295,-1.363846,-0.347154
3,3,0.610625,-0.170428,-1.361771,0.206042,0.623124,0.907441,-0.873814,1.287383,0.901191,...,1.209247,0.095866,-0.287905,-1.110714,-1.660352,0.207231,-0.419119,-0.517563,-1.050697,-0.096327
4,4,-0.689196,-0.858792,0.321308,-0.415649,1.014056,-0.522858,0.926634,-0.390663,0.790054,...,-1.191989,-1.127448,0.246358,0.407769,1.132454,-0.016621,0.964745,0.091532,0.649593,-0.818020


## 2. LASSO Regularisation with K-fold CV

In this section, we begin with LASSO regularisation to reduce our parameter scope using cross-validation.

First, we separate the X predictors from our outcome variable Y.

In [98]:
# Separate the X predictors and Y outcome variable
X0_train = train_df.drop(columns=['Y', 'ID'])
y0_train = train_df['Y'].values
cols = X0_train.columns.tolist()

Second, we then create a matrix of higher order polynomial and interaction terms.

In [99]:
# Define function
def matrix(df, cols):
    # Original terms: X1, X2, X3, ..., X50
    base = df[cols].copy()

    # Squared terms: X1^2, X2^2, X3^2, ..., X50^2
    squares = pd.DataFrame({f"{c}^2": df[c]**2 for c in cols}, index=df.index)

    # Interaction terms: X1*X2, X1*X3, ..., X49*X50
    inter_dict = {}
    for i, c1 in enumerate(cols):
        for j in range(i+1, len(cols)):
            c2 = cols[j]
            inter_dict[f"{c1}*{c2}"] = df[c1] * df[c2]
    inter = pd.DataFrame(inter_dict, index=df.index)

    # Combine all together
    X_vars = pd.concat([base, squares, inter], axis=1)
    return X_vars

# Create the polynomial and interaction terms training set
X_vars_train = matrix(train_df, cols)
X_vars_test = matrix(test_df.drop(columns=['ID']), cols)

print("Shape of original training set:", X0_train.shape)
print("Shape of new training set:", X_vars_train.shape)

Shape of original training set: (2400, 50)
Shape of new training set: (2400, 1325)


Third, we scale the predictors to ensure comparability.

In [100]:
scaler = StandardScaler()
X1_train = scaler.fit_transform(X_vars_train.values)

Fourth, we select the 'optimal' alpha for LASSO regularisation (which we combine with cross-validation).

In [101]:
lasso_cv_model = LassoCV(alphas=np.logspace(-5, 2, num=100), cv=10, max_iter=10000)

# Fit the LassoCV model to the scaled training data
lasso_cv_model.fit(X1_train, y0_train)

# The optimal alpha selected by cross-validation
optimal_alpha_lasso = lasso_cv_model.alpha_
print("Optimal Alpha:", optimal_alpha_lasso)

Optimal Alpha: 0.06579332246575675


As we are interested in a range of models, and want to see which set of models perform the best, let us attempt to select a set of alphas close to our optimal point so we can generate a set of alphas that select us a range of best features given some alpha close to the optimal alpha.

In [102]:
# Get the mean CV error per alpha
mean_mse = lasso_cv_model.mse_path_.mean(axis=1)

# Calculate the standard error of the mean CV error
std_mse = lasso_cv_model.mse_path_.std(axis=1) / np.sqrt(lasso_cv_model.mse_path_.shape[1])

# Index of best alpha
optimal_alpha_lasso_index = np.where(lasso_cv_model.alphas_ == optimal_alpha_lasso)[0][0]

# Create a threshold using one standard error from the mean CV error of the optimal alpha
alpha_threshold = mean_mse[optimal_alpha_lasso_index] + std_mse[optimal_alpha_lasso_index]

# Apply this threshold to select a set of alphas
threshold_inequality = mean_mse <= alpha_threshold
selected_alphas = lasso_cv_model.alphas_[threshold_inequality]

# Use LASSO on these set of alphas to select a set of features for our multiple linear regression model
LASSO_features = {}
for a in selected_alphas:
    model = Lasso(alpha=a).fit(X1_train, y0_train)
    LASSO_features[a] = model

## 3. Model Averaging and Evaluation

In this section, we attempt to select which combination of multiple linear regression models performs best on the training data. The average of the model combinations that perform the best, will be selected for our model prediction.

In [103]:
# Create 10 fold CV to evaluate each model combination
kf = KFold(n_splits=10, shuffle=True, random_state=0)
model_names = list(selected_predictors.keys())
cv_mse = {}

# Loop through all combinations of models
for r in range(1, len(model_names) + 1):
    for combo in combinations(model_names, r):
        fold_mse = []
        for train_idx, val_idx in kf.split(X_vars_train):
            y_train, y_val = y0_train[train_idx], y0_train[val_idx]
            preds = []
            for name in combo:
                X_train = X_vars_train.iloc[train_idx][selected_predictors[name]]
                X_val   = X_vars_train.iloc[val_idx][selected_predictors[name]]
                m = LinearRegression().fit(X_train, y_train)
                preds.append(m.predict(X_val))
            avg_pred = np.column_stack(preds).mean(axis=1)
            fold_mse.append(mean_squared_error(y_val, avg_pred))
        cv_mse[combo] = np.mean(fold_mse)
sorted_cv_mse = sorted(cv_mse.items(), key=lambda x: x[1])
cv_mse_df = pd.DataFrame(sorted_cv_mse, columns=["Models", "CV_MSE"])
cv_mse_df

,Models,CV_MSE
0,"(Model 1, Model 4, Model 5)",3.681728
1,"(Model 1, Model 2, Model 4, Model 5)",3.689367
2,"(Model 1, Model 3, Model 4, Model 5)",3.691211
3,"(Model 1, Model 3, Model 5)",3.694961
4,"(Model 1, Model 2, Model 3, Model 4, Model 5)",3.697348
5,"(Model 2, Model 4, Model 5)",3.699676
6,"(Model 1, Model 5)",3.700377
7,"(Model 2, Model 5)",3.705911
8,"(Model 2, Model 3, Model 4, Model 5)",3.710366
9,"(Model 1, Model 2, Model 3, Model 5)",3.711727


Given that the combination of Model 1, 4, and 5 performs the best. We will use this combination for our final prediction set.

## 4. Model Prediction

In this section, we will average the predictions from Model 1, 4, and 5 for our test set.

In [104]:
models_to_use = ["Model 1", "Model 4", "Model 5"]
preds_test = []

for name in models_to_use:
    cols = selected_predictors[name]
    m = LinearRegression().fit(X_vars_train[cols], y0_train)   # fit on full train
    preds_test.append(m.predict(X_vars_test[cols]))  
y_pred_test = np.column_stack(preds_test).mean(axis=1)

submission_ass3_url = "https://github.com/joshcpld/ada/blob/main/Assignment%203/data/submission.csv?raw=true"
submission_ass3 = pd.read_csv(submission_ass3_url)
submission_ass3 = pd.DataFrame({'ID': test_df['ID'], 'Y': y_pred_test})
submission_ass3.to_csv('submission_ass3_group9.csv', index=False)